In [1]:
import os
import glob

import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns


import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


import pyhrv.time_domain as td
import heartpy as hp
from pyhrv import time_domain
# from pyhrv import hrv
import pyhrv 

%matplotlib agg

In [2]:
os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Code/Swell/hrv dataset/data/raw/rri')

In [3]:
with open('p1.txt') as f:
    lines = f.readlines()
    x = np.array([line.split()[0] for line in lines])
    y = np.array([line.split()[1] for line in lines])

In [4]:
rr = y.astype(np.float64)
t = x.astype(np.float64)

end = t[-1]
min = end/130

hr = 60000/rr
hr_mean = np.mean(hr[0:235])
results = td.rmssd(rr[0:235])

print(hr_mean)
print(results[0])

83.57629084700456
9.368697454179117


In [5]:
# tolerance = 0.0005  # Adjust this value based on the precision you need

# for i in range(2, len(rr)):
#     result = td.rmssd(rr[0:i])
    
#     if abs(result[0] - 15.5545) < tolerance:
#         print("train")
#         print(i)
#     elif abs(result[0] - 12.361) < tolerance:
#         print("test")
#         print(i)

In [6]:
# tolerance = 0.001  # Adjust this value based on the precision you need

# for i in range(1, len(rr)):
#     result = np.mean(hr[0:i])
    
#     if abs(result - 69.4999521131442) < tolerance:
#         print("train")
#         print(i)
#     elif abs(result - 84.1218683005978) < tolerance:
#         print("test")
#         print(i)

In [7]:
files = glob.glob('*.txt')
total_length=0
total_sec=0

for file in files:
    print(file)
    with open(file) as f:
        lines = f.readlines()
        x = np.array([line.split()[0] for line in lines])
        y = np.array([line.split()[1] for line in lines])

    rr = y.astype(np.float64)
    t = x.astype(np.float64)
    length= int(len(t))
    # print(length)
    start = t[0]
    end = t[-1]
    # print(end)
    # min = end/130
    total_sec += end-start
    print('seconds', end-start)
    print('length ', length)
    hr = 60000/rr
    # hr_mean = np.mean(hr[0:333])
    # results = td.rmssd(rr[0:333])

    # print(hr_mean)
    # print(results[0])

    tolerance = 0.0005  # Adjust this value based on the precision you need

    # for i in range(2, len(rr)):
    #     rmssd = td.rmssd(rr[0:i])
    #     hr_mean = np.mean(hr[0:i])
        

    #     if (abs(rmssd[0] - 15.5545) < tolerance) and (abs(hr_mean - 69.4999521131442) < tolerance):
    #         print("train")
    #         print(i)
    #     elif (abs(rmssd[0] - 12.361) < tolerance) and (abs(hr_mean - 84.1218683005978) < tolerance):
    #         print("test")
    #         print(i)
            
    total_length += length

print(total_sec)
print(total_length)

p1.txt
seconds 8993.249975
length  35974
8993.249975
35974


In [8]:
hr_mean = np.mean(hr[0:3])
hr_mean

68.04195682059863

In [9]:
# Load your data from the txt file
# data = pd.read_csv('p1.txt', delimiter='\t', header=None, names=['time_elapsed', 'ibi'])
with open('p1.txt') as f:
    lines = f.readlines()
    time_elapsed = np.array([line.split()[0] for line in lines])
    ibi = np.array([line.split()[1] for line in lines])
time_elapsed = time_elapsed.astype(np.float64)
ibi = ibi.astype(np.float64)


# data_dict = {'time_elapsed': time_elapsed[2395:], 'ibi': ibi[2395:]}
data_dict = {'time_elapsed': time_elapsed, 'ibi': ibi}
data = pd.DataFrame(data_dict)
# print(data['time_elapsed'])
data['time_elapsed'] = np.array(data['time_elapsed']/60)


# Assuming your time_elapsed is in seconds
fs = 1 / np.mean(np.diff(data['time_elapsed']))


# nni = pyhrv.utils.load_sample_nni()

# Function to compute HRV features on a 5-minute moving window
def compute_hrv_features(ibis):
    # hrv_sample = time_domain.nni_parameters(ibis)
    hrv_result = pyhrv.hrv(nni=ibis, show=False)
    # return hrv_sample['nni_mean']
    mean_rr = hrv_result['nni_mean']
    mean_hr = hrv_result['hr_mean']
    sdnn = hrv_result['sdnn']
    sdsd = hrv_result['sdsd']
    pnn50 = hrv_result['pnn50']

    # return hrv['nni_mean','hr_mean', 'sdnn', 'sdsd', 'pnn50']
    return mean_rr, mean_hr, sdnn, sdsd, pnn50


# Initialize variables
# window_size = int(fs) * 5  # 5 minutes in minuttes
# num_samples = len(data['ibi'])
# hrv_samples = []

# # Iterate through the IBI array with a 5-minute moving window
# for i in range(num_samples - window_size + 1):
#     window = data.iloc[i:i+window_size]
#     window_data=window['ibi']
#     window_time = window['time_elapsed']

#     # print(window_time)
#     # print(window_data.values)

hrv_sample = compute_hrv_features(data['ibi'])
    # hrv_samples.append(hrv_sample)


print(hrv_sample)
# Display or save the HRV samples as needed
# for i, hrv_sample in enumerate(hrv_samples):
#     print(f'HRV Sample {i+1}: {hrv_sample}')


# print('Mean all HRV ', np.mean(hrv_samples))


c:\Users\Olsen\anaconda3\Lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '


(952.7958851859677, 64.00781467067293, 132.5863284606889, 7.908501858176414, 0.144552859088761)


In [10]:
# # Set a constant heart rate and introduce some random variability
# heart_rate = 60  # beats per minute
# random_variation = np.random.normal(loc=0, scale=0.1, size=1000)  # random noise

# # Calculate the interbeat intervals (IBI) in seconds
# rr_intervals = 60 / (heart_rate + random_variation)

# # Generate time_elapsed values in seconds
# time_elapsed = np.cumsum(rr_intervals)

# # Create a DataFrame with time_elapsed and ibi columns
# ibi_data = pd.DataFrame({'time_elapsed': time_elapsed, 'ibi': rr_intervals})

# # Save the synthetic IBI data to a text file
# ibi_data.to_csv('synthetic_ibi_data.txt', sep='\t', index=False, header=False)

# # Display the first few rows of the generated IBI data
# print(ibi_data.head())


In [11]:
os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Code/Swell/hrv dataset/data/raw/labels')

def count_conditions(file_path):
    # Read the Excel workbook
    xls = pd.ExcelFile(file_path)
    R_total=0
    N_total=0
    I_total=0
    T_total=0
    # Dictionary to store counts for each sheet
    counts_dict = {}

    # Iterate through each sheet in the workbook
    for sheet_name in xls.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(file_path, sheet_name)

        # Count occurrences of 'R', 'N', and 'I' in the 'Condition' column
        condition_counts = df['Condition'].value_counts()
        R_total = R_total + condition_counts.get('R', 0)
        N_total = N_total + condition_counts.get('N', 0)
        I_total = I_total + condition_counts.get('I', 0)
        T_total = T_total + condition_counts.get('T', 0)
        # Store the counts in the dictionary
        counts_dict[sheet_name] = {
            'R': condition_counts.get('R', 0),
            'N': condition_counts.get('N', 0),
            'I': condition_counts.get('I', 0),
            'T': condition_counts.get('T', 0),
            'R_total': R_total,
            'N_total': N_total,
            'I_total': I_total,
        }

    return counts_dict

# Replace 'your_excel_file.xlsx' with the path to your Excel workbook
file_path = 'hrv stress labels.xlsx'
result = count_conditions(file_path)
# total_r = 0
# total_n = 0
# total_i = 0
# total_t = 0
# # Print the result
# for sheet, counts in result.items():
#     print(f"Sheet: {sheet}")
#     print(f"  R count: {counts['R']}")
#     print(f"  N count: {counts['N']}")
#     print(f"  I count: {counts['I']}")
#     print(f"  T count: {counts['T']}\n")
#     total_r += counts['R']
#     total_n += counts['N']
#     total_i += counts['I']
#     total_t += counts['T']

# print("____________________________________")


# # for sheet, counts in result.items():
# #     print(f"Sheet: {sheet}")
# #     print(f"  Total R count: {counts['R_total']}")
# #     print(f"  Total N count: {counts['N_total']}")
# #     print(f"  Total I count: {counts['I_total']}\n")

# print(total_r)
# print(total_n)
# print(total_i)
# print(total_t)
# print("____________________________________")
# print(total_r+total_n+total_i+total_t)
# print("____________________________________")
# # Sheet: p25
#   Total R count: 451
#   Total N count: 1029
#   Total I count: 996

In [12]:
os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Code/Swell/hrv dataset/data/raw/labels')


# Replace 'your_excel_file.xlsx' with the actual path to your Excel workbook
file_path = 'hrv stress labels.xlsx'

# Create an empty DataFrame to store the consolidated data
consolidated_df = pd.DataFrame(columns=['time_elapsed', 'condition', 'subject'])

# Read all sheets in the Excel workbook
excel_sheets = pd.read_excel(file_path, sheet_name=None)

# Iterate through each sheet and extract required columns
for sheet_name, sheet_data in excel_sheets.items():
    # Assuming the columns you need are named 'time_elapsed', 'condition', and 'subject'
    required_columns = ['ElapsedTime', 'Condition', 'subject']

    # Check if all required columns exist in the sheet
    if all(col in sheet_data.columns for col in required_columns):
        # Extract the required columns and append to the consolidated DataFrame
        sheet_subset = sheet_data[required_columns]
        consolidated_df = pd.concat([consolidated_df, sheet_subset], ignore_index=True)

# Print the consolidated DataFrame
print(consolidated_df)


     time_elapsed condition subject  ElapsedTime Condition
0             NaN       NaN      p1          0.0         R
1             NaN       NaN      p1          1.0         R
2             NaN       NaN      p1          2.0         R
3             NaN       NaN      p1          3.0         R
4             NaN       NaN      p1          4.0         R
...           ...       ...     ...          ...       ...
3135          NaN       NaN     p25        170.0         T
3136          NaN       NaN     p25        171.0         T
3137          NaN       NaN     p25        172.0         T
3138          NaN       NaN     p25        173.0         T
3139          NaN       NaN     p25        174.0         T

[3140 rows x 5 columns]


In [14]:
import numpy as np
import pandas as pd
import heartpy as hp
from pyhrv import time_domain
from pyhrv import hrv
import pyhrv

# Assuming you already have the consolidated_df DataFrame from the previous example
# Make sure to adjust the path to your folder containing TXT files
folder_path = '../rri/'

# Function to compute HRV features on a 5-minute moving window
def compute_hrv_features(ibis):
    hrv_sample = time_domain.nni_parameters(ibis)
    return hrv_sample['nni_mean']


# Sliding window function (you can replace this with your custom function)
def sliding_window(data, window_size):
    # Initialize variables
    # window_size = int(fs) * 5  # 5 minutes in minuttes
    num_samples = len(data['ibi'])
    hrv_samples = []

    # Iterate through the IBI array with a 5-minute moving window
    for i in range(num_samples - window_size + 1):
        window = data.iloc[i:i+window_size]
        window_data=window['ibi']
        window_time = window['time_elapsed']

        # print(window_time)
        # print(window_data.values)
        hrv_sample = compute_hrv_features(window_data)
        hrv_samples.append(hrv_sample)
    return hrv_samples


window_size = 4 * 5  # 5 minutes in minuttes
all_hrv_samples = []

# Iterate through all TXT files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        subject_id = os.path.splitext(filename)[0]
        txt_file_path = os.path.join(folder_path, filename)
        print(filename)
        
        # Read content from the TXT file and parse timestamps and IBI values
        # txt_data = pd.read_csv(txt_file_path, sep='\t')  # Assuming tab-separated values
        with open(txt_file_path) as f:
            lines = f.readlines()
            timestamp = np.array([line.split()[0] for line in lines])
            ibi = np.array([line.split()[1] for line in lines])
        
        timestamp = timestamp.astype(np.float64)
        ibi = ibi.astype(np.float64)
        timestamp = timestamp.astype(np.int64)

        data_dict = {'timestamp': timestamp[2395:]/60, 'ibi': ibi[2395:]}
        txt_data = pd.DataFrame(data_dict)
        
        # Merge with the consolidated_df DataFrame based on 'time_elapsed'
        merged_data = pd.merge(consolidated_df, txt_data, left_on='ElapsedTime', right_on='timestamp')

        # Filter rows based on condition 'R' and subject
        filtered_data = merged_data[(merged_data['Condition'] != 'R') & (merged_data['subject'] == subject_id)]

        # Iterate through the timestamps and apply the sliding window function
        for timestamp in txt_data['timestamp']:
            # Filter data based on timestamp
            window_data = filtered_data[(filtered_data['timestamp'] >= timestamp) & (filtered_data['timestamp'] <= timestamp + window_size)]

            # Apply the sliding window function
            result = sliding_window(window_data, window_size)

            if len(result) > 0:
                # if not result[0] != (all_hrv_samples[-1][-1] if all_hrv_samples else None):
                    # all_hrv_samples.append(result[0])

                all_hrv_samples.append(result[0])
            # Do something with the result, e.g., print or save it
            # print(result)

print(all_hrv_samples)

p1.txt
[807.5628839999999, 807.5628839999999, 807.5628839999999, 807.5628839999999, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.8433105, 859.84

In [15]:
unique_set = set(all_hrv_samples)

# Count unique values

unique_count = len(unique_set)
# Output
print("Count of unique values using a set:", unique_count)

Count of unique values using a set: 70


In [16]:
import numpy as np
import pandas as pd
import heartpy as hp
from pyhrv import time_domain
from pyhrv import hrv
import pyhrv


import os
os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Code/Swell/hrv dataset/data/raw/rri')


# Load your data from the txt file
# data = pd.read_csv('p1.txt', delimiter='\t', header=None, names=['time_elapsed', 'ibi'])
with open('p1.txt') as f:
    lines = f.readlines()
    time_elapsed = np.array([line.split()[0] for line in lines])
    ibi = np.array([line.split()[1] for line in lines])
    
time_elapsed = time_elapsed.astype(np.float64)
ibi = ibi.astype(np.float64)


data_dict = {'time_elapsed': time_elapsed, 'ibi': ibi}
data = pd.DataFrame(data_dict)
# print(data['time_elapsed'])
data['time_elapsed'] = np.array(data['time_elapsed']/60)


# Assuming your time_elapsed is in seconds
fs = 1 / np.mean(np.diff(data['time_elapsed']))


# nni = pyhrv.utils.load_sample_nni()

# Function to compute HRV features on a 5-minute moving window
def compute_hrv_features(ibis):
    hrv_sample = time_domain.nni_parameters(ibis)
    return hrv_sample['nni_mean'], hrv_sample['']


# Initialize variables
window_size = int(fs) * 5  # 5 minutes in minuttes
num_samples = len(data['ibi'])
hrv_samples = []


# Iterate through the IBI array with a 5-minute moving window
for i in range(num_samples - window_size + 1):
    window = data.iloc[i:i+window_size]
    window_data=window['ibi']
    window_time = window['time_elapsed']

    # print(window_time)
    # print(window_data.values)
    hrv_sample = compute_hrv_features(window_data)
    hrv_samples.append(hrv_sample)


# Display or save the HRV samples as needed
for i, hrv_sample in enumerate(hrv_samples):
    print(f'HRV Sample {i+1}: {hrv_sample}')


print('Mean all HRV ', np.mean(hrv_samples))


HRV Sample 1: 932.17644735
HRV Sample 2: 932.2243680250001
HRV Sample 3: 932.2637822583333
HRV Sample 4: 932.3073502166666
HRV Sample 5: 932.3648073833333
HRV Sample 6: 932.4429737916666
HRV Sample 7: 932.5460692333334
HRV Sample 8: 932.676100175
HRV Sample 9: 932.8330080333333
HRV Sample 10: 933.0149853833334
HRV Sample 11: 933.2175936250001
HRV Sample 12: 933.4324725166667
HRV Sample 13: 933.6490518833334
HRV Sample 14: 933.8551766083333
HRV Sample 15: 934.0389593083333
HRV Sample 16: 934.1938924833335
HRV Sample 17: 934.3194357083335
HRV Sample 18: 934.4209170916666
HRV Sample 19: 934.5056390750001
HRV Sample 20: 934.5775019749999
HRV Sample 21: 934.6366219666667
HRV Sample 22: 934.6808753083334
HRV Sample 23: 934.7096892166668
HRV Sample 24: 934.7248446083332
HRV Sample 25: 934.7297834916667
HRV Sample 26: 934.7274989083334
HRV Sample 27: 934.7204281416667
HRV Sample 28: 934.7105953083335
HRV Sample 29: 934.7004456833333
HRV Sample 30: 934.6931031000001
HRV Sample 31: 934.692354233

In [22]:
# # Import packages
# import biosppy
# import pyhrv.time_domain as td

# # Load sample ECG signal
# signal = np.loadtxt('SampleECG.txt')[:, -1]

# # Get R-peaks series using biosppy
# t, filtered_signal, rpeaks = biosppy.signals.ecg.ecg(signal)[:3]
# peaks=[]
# print(rpeaks)

# for i in range(1, len(rpeaks)):
#     peaks.append(rpeaks[i]-rpeaks[i-1])

# print(peaks)

# # Compute parameters using R-peak series
# results = td.nni_parameters(rpeaks=t[rpeaks])

c:\Users\Olsen\anaconda3\Lib\site-packages\biosppy\plotting.py:1538: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


[  670  1423  2188  2941  3676  4429  5198  5988  6776  7567  8339  9084
  9800 10518 11252 12022 12859 13728 14596 15446 16258 17017 17759 18510
 19270 20039 20810 21556]
[753, 765, 753, 735, 753, 769, 790, 788, 791, 772, 745, 716, 718, 734, 770, 837, 869, 868, 850, 812, 759, 742, 751, 760, 769, 771, 746]
